<a href="https://colab.research.google.com/github/osmarbraz/exemplos_Llama/blob/main/Exemplos_Chunk_Overlap_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exemplo de divisão e sobreposiçao de texto usando Langchain

Cria a divisão de um texto em pedaços(chunks) por tokens e caracteres para submeter a um LLM. Realiza a sobreposição(overlap) de parte do texto para manter a semântica entre os chunks.

Semelhante a uma janela deslizante.

Teste online da chunk(divisão) e overlap(overlap) usando um arquivo texto: https://chunkerizer.streamlit.app/

Entendendo o chunk e overlap
https://gustavo-espindola.medium.com/chunk-division-and-overlap-understanding-the-process-ade7eae1b2bd

LangChain: como dividir corretamente seus chunks(pedaços):
https://www.youtube.com/watch?v=n0uPzvGTFI0

Estratégias de chucking: https://www.pinecone.io/learn/chunking-strategies/

Como agrupar dados de texto - uma análise comparativa: https://towardsdatascience.com/how-to-chunk-text-data-a-comparative-analysis-3858c4a0997a

# 0 - Preparação do ambiente
Preparação do ambiente para execução do exemplo.

## Tratamento de logs

Método para tratamento dos logs.

In [61]:
# Biblioteca de logging
import logging

# Formatando a mensagem de logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)

## Identificando o ambiente Colab

Cria uma variável para identificar que o notebook está sendo executado no Google Colaboratory.

In [62]:
# Se estiver executando no Google Colaboratory
import sys

# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = "google.colab" in sys.modules

## Funções auxiliares

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [63]:
# Import das bibliotecas.
import time
import datetime

def formataTempo(tempo):
    """
      Pega a tempo em segundos e retorna uma string hh:mm:ss
    """
    # Arredonda para o segundo mais próximo.
    tempo_arredondado = int(round((tempo)))

    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempo_arredondado))

# 1 - Instalação das bibliotecas

Biblioteca para manipular pdf

https://pypi.org/project/pypdf/

In [64]:
!pip install pypdf==3.16.4

Biblioteca que persiste os embeddings e realiza busca semântica.

https://pypi.org/project/chromadb/

In [65]:
!pip install chromadb==0.4.15

Biblioteca para realizar a divisão por token.

In [66]:
!pip install tiktoken==0.5.1

Bibioteca LangChain é um framework de código aberto para o desenvolvimento de aplicações usando modelos de linguagem grandes.

https://pypi.org/project/langchain/

In [67]:
!pip install langchain==0.0.323

# 2 - Tipos de divisão

https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed

In [68]:
#Import de bibliotecas
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter, TokenTextSplitter

chunk_size =26
chunk_overlap = 4

r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)
c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

t_splitter = TokenTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

## Divisor recursivo de caracteres

In [69]:
# Divisor recursivo do texto
text1 = 'abcdefghijklmnopqrstuvwxyz'
r_splitter.split_text(text1)
# Saída - ['abcdefghijklmnopqrstuvwxyz']

['abcdefghijklmnopqrstuvwxyz']

In [70]:
# Dividor de caracter do texto
text2 = 'abcdefghijklmnopqrstuvwxyzabcdefg'
r_splitter.split_text(text2)
# Saída - ['abcdefghijklmnopqrstuvwxyz', 'wxyzabcdefg']

['abcdefghijklmnopqrstuvwxyz', 'wxyzabcdefg']

In [71]:
# Dividor recursivo de texto
text3 = "a b c d e f g h i j k l m n o p q r s t u v w x y z"
r_splitter.split_text(text3)
# Saída - ['a b c d e f g h i j k l m', 'l m n o p q r s t u v w x', 'w x y z']

['a b c d e f g h i j k l m', 'l m n o p q r s t u v w x', 'w x y z']

## Divisor de caracter

In [72]:
# Dividor de caracter texto
c_splitter.split_text(text3)
# Saída - ['a b c d e f g h i j k l m n o p q r s t u v w x y z']

['a b c d e f g h i j k l m n o p q r s t u v w x y z']

In [73]:
# Character Text Splitter with separator defined
# Divisor de caracter texto com separador definido
c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separator = ' '
)
c_splitter.split_text(text3)
# Saída - ['a b c d e f g h i j k l m', 'l m n o p q r s t u v w x', 'w x y z']

['a b c d e f g h i j k l m', 'l m n o p q r s t u v w x', 'w x y z']

## Divisor de token

In [74]:
# Dividor de token do texto
t_splitter.split_text(text1)
# Saída - ['abcdefghijklmnopqrstuvwxyz']

['abcdefghijklmnopqrstuvwxyz']

In [75]:
# Dividor de token do texto
text4 = 'ab cd ef gh ij kl mn op qr st uv wx yz ab cd ef gh ij kl mn op qr st uv wx yz'
t_splitter.split_text(text4)
# Saída - ['ab cd ef gh ij kl mn op qr st uv wx yz ab cd ef gh',' cd ef gh ij kl mn op qr st uv wx yz']

['ab cd ef gh ij kl mn op qr st uv wx yz ab cd ef gh',
 ' cd ef gh ij kl mn op qr st uv wx yz']

# 3 - Divisão por Token

## 3.1 - Manipulando documentos Texto

### Cria o texto

In [76]:
# trecho de https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', #As Vítimas algozes


texto = """
No interior e principalmente longe da vila, ou da freguesia e dos povoados há quase
sempre uma venda perto da fazenda: é a parasita que se apega à árvore; pior que isso,
é a inimiga hipócrita que rende vassalagem à sua vítima.
A venda de que falo é uma taberna especialíssima
que não poderia existir, manter-se, medrar em outras condições locais, e em
outras condições do trabalho rural, e nem se confunde com a taberna regular que
em toda parte se encontra, quanto mais com as casas de grande ou pequeno comércio,
onde os lavradores ricos e pobres se provêem do que precisa a casa, quando não
lhes é possível esperar pelas remessas dos seus consigna­tários ou fregueses.
Essa parasita das fazendas e estabelecimentos agrícolas
das vizinhanças facilmente se pode conhecer por suas feições e modos
característicos, se nos é lícito dizer assim: uma se parece com todas e não há
hipótese em que alguma delas, por mais dissimulada que seja, chegue a perder o
cará­ter da família.
É uma pequena casa de taipa e coberta de telha, tendo às
vezes na frente varanda aberta pelos três lados, também coberta de telha e com
o teto sustido por esteios fortes, mas rudes e ainda mesmo tortos; as paredes
nem sempre são caiadas, o chão não tem assoalho nem ladrilho; quando há
varanda, abrem-se para ela uma porta e uma janela; dentro está a ven­da: entre
a porta e a janela encostado à parede um banco de pau, defronte um balcão tosco
e no bojo ou no espaço que se vê além, grotesca armação de tábuas contendo
garrafas, botijas, latas de tabaco em pó, a um canto algumas voltas de fumo em
rolo e uma ruim manta de carne-seca. Eis a venda.
Há muitas que nem chegam à opulência da que aí fica
descrita; em todas porém aparece humilde no fundo do quase vazio bojo a porta
baixa que comunica pelo corredor imundo com dois ou mais quartos escuros, onde
se recolhem as pingues colheitas agrícolas do vendelhão que aliás não tem
lavoura.
"""

### Divide e sobrepõe o texto

In [77]:
# Import das bibliotecas
import os
import time
from langchain.text_splitter import TokenTextSplitter

# Parâmetros
chunk_tamanho = 500
chunk_sobreposicao = 100

text_splitter = TokenTextSplitter(
    chunk_size = chunk_tamanho,
    chunk_overlap  = chunk_sobreposicao, # Número de tokens sobrepostos entre chunks(pedaços)
    length_function = len, # Usa o comprimento do texto como medida de tamanho
    add_start_index = True, #
)

# Guarda o tempo de início
tempo_inicio = time.time()

# Calcula os chunks dos textos
chunks = text_splitter.create_documents([texto])

tempo_final = time.time()

print(f"Carregando e dividindo {len(texto)} documentos texto em {tempo_final - tempo_inicio} segundos!")

Carregando e dividindo 1918 documentos texto em 0.0031900405883789062 segundos!


In [78]:
print(len(chunks))

2


In [79]:
print(chunks)

[Document(page_content='\nNo interior e principalmente longe da vila, ou da freguesia e dos povoados há quase\nsempre uma venda perto da fazenda: é a parasita que se apega à árvore; pior que isso,\né a inimiga hipócrita que rende vassalagem à sua vítima.\nA venda de que falo é uma taberna especialíssima\nque não poderia existir, manter-se, medrar em outras condições locais, e em\noutras condições do trabalho rural, e nem se confunde com a taberna regular que\nem toda parte se encontra, quanto mais com as casas de grande ou pequeno comércio,\nonde os lavradores ricos e pobres se provêem do que precisa a casa, quando não\nlhes é possível esperar pelas remessas dos seus consigna\xadtários ou fregueses.\nEssa parasita das fazendas e estabelecimentos agrícolas\ndas vizinhanças facilmente se pode conhecer por suas feições e modos\ncaracterísticos, se nos é lícito dizer assim: uma se parece com todas e não há\nhipótese em que alguma delas, por mais dissimulada que seja, chegue a perder o\ncar

### Mostra os chunks

A sobreposição se encontra ao final e início de cada pedaço(chunk). A variável 'start_index' define onde começa o texto sem a sobreposição.

In [80]:
for i, chunk in enumerate(chunks):
  if i < 20:
    tokens = chunk.page_content.split(" ")
    print('chunk #',i, len(tokens),' qtde char :', len(chunk.page_content),' qtde token :', ' start_index:', chunk.metadata.get('start_index') )
    print()
    print(chunk.page_content)
    print('-----------------------------------------------------------------------')

chunk # 0 203  qtde char : 1259  qtde token :  start_index: 0


No interior e principalmente longe da vila, ou da freguesia e dos povoados há quase
sempre uma venda perto da fazenda: é a parasita que se apega à árvore; pior que isso,
é a inimiga hipócrita que rende vassalagem à sua vítima.
A venda de que falo é uma taberna especialíssima
que não poderia existir, manter-se, medrar em outras condições locais, e em
outras condições do trabalho rural, e nem se confunde com a taberna regular que
em toda parte se encontra, quanto mais com as casas de grande ou pequeno comércio,
onde os lavradores ricos e pobres se provêem do que precisa a casa, quando não
lhes é possível esperar pelas remessas dos seus consigna­tários ou fregueses.
Essa parasita das fazendas e estabelecimentos agrícolas
das vizinhanças facilmente se pode conhecer por suas feições e modos
característicos, se nos é lícito dizer assim: uma se parece com todas e não há
hipótese em que alguma delas, por mais dissimulada que seja,

## 3.2 - Manipulando Documentos PDF

https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf

### - Link dos PDFs

In [81]:
# As Vítimas Algozes
url1 = 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116977'

# A Escrava Isaura
url2 = 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=92390'

### Download dos pdfs

In [82]:
# Import das bibliotecas
import subprocess

# As Vítimas Algozes

# Arquivo de destino1
destino1 = 'arquivo1.pdf'

# Executa o comando wget no prompt
subprocess.call(["wget", url1, "-O", destino1])

# Arquivo de destino1
destino2 = 'arquivo2.pdf'

# Executa o comando wget no prompt
subprocess.call(["wget", url1, "-O", destino2])

0

### Carrega os pdfs

In [83]:
# Import das bibliotecas
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

# Define o diretório
diretorio = '/content'

 # Cria o carregador do PDF
carregador = PyPDFDirectoryLoader(diretorio)

# Carrega os documentos
documentos = carregador.load()

### Divide e sobrepõe os pdfs

In [84]:
# Import das bibliotecas
import os
import time
from langchain.text_splitter import TokenTextSplitter

# Parâmetros
chunk_tamanho = 500
chunk_sobreposicao = 100

text_splitter = TokenTextSplitter(
    chunk_size = chunk_tamanho,
    chunk_overlap  = chunk_sobreposicao, # Número de tokens sobrepostos entre chunks(pedaços)
    length_function = len, # Usa o comprimento do texto como medida de tamanho
    add_start_index = True, #
)

# Guarda o tempo de início
tempo_inicio = time.time()

# Calcula os chunks dos textos
chunks = text_splitter.split_documents(documentos)

tempo_final = time.time()

print(f"Carregando e dividindo {len(texto)} documentos pdfs em {tempo_final - tempo_inicio} segundos!")

Carregando e dividindo 1918 documentos pdfs em 0.5524847507476807 segundos!


### Mostra os chunks

A sobreposição se encontra ao final e início de cada pedaço(chunk). A variável 'start_index' define onde começa o texto sem a sobreposição.

In [85]:
for i, chunk in enumerate(chunks):
  if i < 20:
    tokens = chunk.page_content.split(" ")
    print('chunk #',i, len(tokens),' qtde char :', len(chunk.page_content),' qtde token :', ' start_index:', chunk.metadata.get('start_index') )
    print()
    print(chunk.page_content)
    print('-----------------------------------------------------------------------')

chunk # 0 47  qtde char : 376  qtde token :  start_index: 0

J MAS-ALGOZE 
ROMAM 
~% 
LIVRARIA DE B. L. GARNIER 
«9, rua do Ouvidor, CO 
Grandesortimento del.ivros clássicos, Medicina, 
Sciencias e Arlís, Junsprudcncia, Littsratura, 
Novellas, lllustrações, Educação, Devoção, Atlas, 
Happas geographicos, etc, etc. 
Livros francezes, portuguezes, inqlezes, italianos, ele-, 
Encarrega-se áe qualquer conmissio de Liuros-
BIO DE JIMIIIO 
-----------------------------------------------------------------------
chunk # 1 14  qtde char : 79  qtde token :  start_index: 0

Ie ne fay rien 
sans 
Gayeté 
(Montaigne, Des livres) 
Ex Libris 
José Mindlin 
-----------------------------------------------------------------------
chunk # 2 3  qtde char : 20  qtde token :  start_index: 0

AS VICTIMAS-ALGOZES 
-----------------------------------------------------------------------
chunk # 3 26  qtde char : 155  qtde token :  start_index: 0

AS VICTIMAS-ALGOZES 
QUADROS DA ESCRAVIDÃO 
ROMANCES 
POR 
JOAQUI

## 3.3 - Manipulando Documentos HTML

https://python.langchain.com/docs/integrations/document_loaders/web_base

### Links dos HTML

In [86]:
# As Vítimas Algozes
url1 = "https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979"

# A Escrava Isaura
url2 = "https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=92389"

### Carrega os HTMLs

Se ocorrer o **erro** abaixo, instale o Chromedb. Alguma dependência do pacote resolve o problema.

---------------------------------------------------------------------------
IncompleteRead                            Traceback (most recent call last)
/usr/local/lib/python3.10/dist-packages/urllib3/response.py in _error_catcher(self)
    709             try:
--> 710                 yield
    711

15 frames
IncompleteRead: IncompleteRead(821958 bytes read, 2240625 more expected)

The above exception was the direct cause of the following exception:

ProtocolError                             Traceback (most recent call last)
ProtocolError: ('Connection broken: IncompleteRead(821958 bytes read, 2240625 more expected)', IncompleteRead(821958 bytes read, 2240625 more expected))

During handling of the above exception, another exception occurred:

ChunkedEncodingError                      Traceback (most recent call last)
/usr/local/lib/python3.10/dist-packages/requests/models.py in generate()
    816                     yield from self.raw.stream(chunk_size, decode_content=True)
    817                 except ProtocolError as e:
--> 818                     raise ChunkedEncodingError(e)
    819                 except DecodeError as e:
    820                     raise ContentDecodingError(e)

ChunkedEncodingError: ('Connection broken: IncompleteRead(821958 bytes read, 2240625 more expected)', IncompleteRead(821958 bytes read, 2240625 more expected))

In [87]:
# Import das bibliotecas
from langchain.document_loaders import WebBaseLoader

# Cria o carregador da página
carregador = WebBaseLoader([url1, url2])
# carregador.requests_kwargs = {'verify':False}

# Carrega os documentos
documentos = carregador.load()

### Divide e sobrepõe os HTMLs

In [88]:
# Import das bibliotecas
import os
import time
from langchain.text_splitter import TokenTextSplitter

# Parâmetros
chunk_tamanho = 500
chunk_sobreposicao = 100

text_splitter = TokenTextSplitter(
    chunk_size = chunk_tamanho,
    chunk_overlap  = chunk_sobreposicao, # Número de tokens sobrepostos entre chunks(pedaços)
    length_function = len, # Usa o comprimento do texto como medida de tamanho
    add_start_index = True, #
)

# Guarda o tempo de início
tempo_inicio = time.time()

# Calcula os chunks dos textos
chunks = text_splitter.split_documents(documentos)

tempo_final = time.time()

print(f"Carregando e dividindo {len(texto)} documentos htmls em {tempo_final - tempo_inicio} segundos!")

Carregando e dividindo 1918 documentos htmls em 0.3189356327056885 segundos!


### Mostra os chunks

A sobreposição se encontra ao final e início de cada pedaço(chunk). A variável 'start_index' define onde começa o texto sem a sobreposição.

In [89]:
for i, chunk in enumerate(chunks):
  if i < 20:
    tokens = chunk.page_content.split(" ")
    print('chunk #',i, len(tokens),' qtde char :', len(chunk.page_content),' qtde token :', ' start_index:', chunk.metadata.get('start_index') )
    print()
    print(chunk.page_content)
    print('-----------------------------------------------------------------------')

chunk # 0 165  qtde char : 1199  qtde token :  start_index: 0

















As vítimas algozes - Joaquim Manuel de Macedo



Fonte: Biblioteca Digital de Literatura de Países Lusófonos

LITERATURA BRASILEIRA 
Textos literários em
meio eletrônico
As
Vítimas-Algozes, de Joaquim Manuel de Macedo

Edição de base:
Biblioteca Nacional – setor de obras digitalizadas
ÍNDICE
SIMEÃO, O CRIOULO
PAI- RAIOL, O FEITICEIRO
LUCINDA, A
MUCAMA
CONCLUSÃO
I
SIMEÃO, O CRIOULO
I
No interior e principalmente longe da vila, ou da
freguesia e dos povoados há quase sempre uma venda perto da fazenda: é a
parasita que se apega à árvore; pior que isso, é a inimiga hipócrita que rende
vassalagem à sua vítima.
A venda de que falo é uma taberna especialíssima
que não poderia existir, manter-se, medrar em outras condições locais, e em
outras condições do trabalho rural, e nem se confunde com a taberna regular que
em toda parte se encontra, quanto mais com as casas de grande ou pequeno comércio,
onde os lavradores r

# 4 - Divisão Recursiva de Caracter

## 4.1 - Manipulando documentos Texto

### Cria o texto

In [90]:
# trecho de https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', #As Vítimas algozes


texto = """
No interior e principalmente longe da vila, ou da freguesia e dos povoados há quase
sempre uma venda perto da fazenda: é a parasita que se apega à árvore; pior que isso,
é a inimiga hipócrita que rende vassalagem à sua vítima.
A venda de que falo é uma taberna especialíssima
que não poderia existir, manter-se, medrar em outras condições locais, e em
outras condições do trabalho rural, e nem se confunde com a taberna regular que
em toda parte se encontra, quanto mais com as casas de grande ou pequeno comércio,
onde os lavradores ricos e pobres se provêem do que precisa a casa, quando não
lhes é possível esperar pelas remessas dos seus consigna­tários ou fregueses.
Essa parasita das fazendas e estabelecimentos agrícolas
das vizinhanças facilmente se pode conhecer por suas feições e modos
característicos, se nos é lícito dizer assim: uma se parece com todas e não há
hipótese em que alguma delas, por mais dissimulada que seja, chegue a perder o
cará­ter da família.
É uma pequena casa de taipa e coberta de telha, tendo às
vezes na frente varanda aberta pelos três lados, também coberta de telha e com
o teto sustido por esteios fortes, mas rudes e ainda mesmo tortos; as paredes
nem sempre são caiadas, o chão não tem assoalho nem ladrilho; quando há
varanda, abrem-se para ela uma porta e uma janela; dentro está a ven­da: entre
a porta e a janela encostado à parede um banco de pau, defronte um balcão tosco
e no bojo ou no espaço que se vê além, grotesca armação de tábuas contendo
garrafas, botijas, latas de tabaco em pó, a um canto algumas voltas de fumo em
rolo e uma ruim manta de carne-seca. Eis a venda.
Há muitas que nem chegam à opulência da que aí fica
descrita; em todas porém aparece humilde no fundo do quase vazio bojo a porta
baixa que comunica pelo corredor imundo com dois ou mais quartos escuros, onde
se recolhem as pingues colheitas agrícolas do vendelhão que aliás não tem
lavoura.
"""

### Divide e sobrepõe o texto

In [91]:
# Import das bibliotecas
import os
import time
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Parâmetros
chunk_tamanho = 500
chunk_sobreposicao = 100

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_tamanho,
    chunk_overlap  = chunk_sobreposicao, # Número de tokens sobrepostos entre chunks(pedaços)
    length_function = len, # Usa o comprimento do texto como medida de tamanho
    add_start_index = True, #
)

# Guarda o tempo de início
tempo_inicio = time.time()

# Calcula os chunks dos textos
chunks = text_splitter.create_documents([texto])

tempo_final = time.time()

print(f"Carregando e dividindo {len(texto)} documentos texto em {tempo_final - tempo_inicio} segundos!")

Carregando e dividindo 1918 documentos texto em 0.0020737648010253906 segundos!


In [92]:
print(len(chunks))

5


In [93]:
print(chunks)

[Document(page_content='No interior e principalmente longe da vila, ou da freguesia e dos povoados há quase\nsempre uma venda perto da fazenda: é a parasita que se apega à árvore; pior que isso,\né a inimiga hipócrita que rende vassalagem à sua vítima.\nA venda de que falo é uma taberna especialíssima\nque não poderia existir, manter-se, medrar em outras condições locais, e em\noutras condições do trabalho rural, e nem se confunde com a taberna regular que', metadata={'start_index': 1}), Document(page_content='outras condições do trabalho rural, e nem se confunde com a taberna regular que\nem toda parte se encontra, quanto mais com as casas de grande ou pequeno comércio,\nonde os lavradores ricos e pobres se provêem do que precisa a casa, quando não\nlhes é possível esperar pelas remessas dos seus consigna\xadtários ou fregueses.\nEssa parasita das fazendas e estabelecimentos agrícolas\ndas vizinhanças facilmente se pode conhecer por suas feições e modos', metadata={'start_index': 353}

### Mostra os chunks

A sobreposição se encontra ao final e início de cada pedaço(chunk). A variável 'start_index' define onde começa o texto sem a sobreposição.

In [94]:
for i, chunk in enumerate(chunks):
  if i < 20:
    tokens = chunk.page_content.split(" ")
    print('chunk #',i, len(tokens),' qtde char :', len(chunk.page_content),' qtde token :', ' start_index:', chunk.metadata.get('start_index') )
    print()
    print(chunk.page_content)
    print('-----------------------------------------------------------------------')

chunk # 0 72  qtde char : 431  qtde token :  start_index: 1

No interior e principalmente longe da vila, ou da freguesia e dos povoados há quase
sempre uma venda perto da fazenda: é a parasita que se apega à árvore; pior que isso,
é a inimiga hipócrita que rende vassalagem à sua vítima.
A venda de que falo é uma taberna especialíssima
que não poderia existir, manter-se, medrar em outras condições locais, e em
outras condições do trabalho rural, e nem se confunde com a taberna regular que
-----------------------------------------------------------------------
chunk # 1 68  qtde char : 444  qtde token :  start_index: 353

outras condições do trabalho rural, e nem se confunde com a taberna regular que
em toda parte se encontra, quanto mais com as casas de grande ou pequeno comércio,
onde os lavradores ricos e pobres se provêem do que precisa a casa, quando não
lhes é possível esperar pelas remessas dos seus consigna­tários ou fregueses.
Essa parasita das fazendas e estabelecimentos agríco

## 4.2 - Manipulando Documentos PDF

https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf

### - Link dos PDFs

In [95]:
# As Vítimas Algozes
url1 = 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116977'

# A Escrava Isaura
url2 = 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=92390'

### Download dos pdfs

In [96]:
# Import das bibliotecas
import subprocess

# As Vítimas Algozes

# Arquivo de destino1
destino1 = 'arquivo1.pdf'

# Executa o comando wget no prompt
subprocess.call(["wget", url1, "-O", destino1])

# Arquivo de destino1
destino2 = 'arquivo2.pdf'

# Executa o comando wget no prompt
subprocess.call(["wget", url1, "-O", destino2])

0

### Carrega os pdfs

In [97]:
# Import das bibliotecas
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

# Define o diretório
diretorio = '/content'

 # Cria o carregador do PDF
carregador = PyPDFDirectoryLoader(diretorio)

# Carrega os documentos
documentos = carregador.load()

### Divide e sobrepõe os pdfs

In [98]:
# Import das bibliotecas
import os
import time
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Parâmetros
chunk_tamanho = 500
chunk_sobreposicao = 100

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_tamanho,
    chunk_overlap  = chunk_sobreposicao, # Número de tokens sobrepostos entre chunks(pedaços)
    length_function = len, # Usa o comprimento do texto como medida de tamanho
    add_start_index = True, #
)

# Guarda o tempo de início
tempo_inicio = time.time()

# Calcula os chunks dos textos
chunks = text_splitter.split_documents(documentos)

tempo_final = time.time()

print(f"Carregando e dividindo {len(texto)} documentos pdfs em {tempo_final - tempo_inicio} segundos!")

Carregando e dividindo 1918 documentos pdfs em 0.06796383857727051 segundos!


### Mostra os chunks

A sobreposição se encontra ao final e início de cada pedaço(chunk). A variável 'start_index' define onde começa o texto sem a sobreposição.

In [99]:
for i, chunk in enumerate(chunks):
  if i < 20:
    tokens = chunk.page_content.split(" ")
    print('chunk #',i, len(tokens),' qtde char :', len(chunk.page_content),' qtde token :', ' start_index:', chunk.metadata.get('start_index') )
    print()
    print(chunk.page_content)
    print('-----------------------------------------------------------------------')

chunk # 0 46  qtde char : 375  qtde token :  start_index: 0

J MAS-ALGOZE 
ROMAM 
~% 
LIVRARIA DE B. L. GARNIER 
«9, rua do Ouvidor, CO 
Grandesortimento del.ivros clássicos, Medicina, 
Sciencias e Arlís, Junsprudcncia, Littsratura, 
Novellas, lllustrações, Educação, Devoção, Atlas, 
Happas geographicos, etc, etc. 
Livros francezes, portuguezes, inqlezes, italianos, ele-, 
Encarrega-se áe qualquer conmissio de Liuros-
BIO DE JIMIIIO
-----------------------------------------------------------------------
chunk # 1 13  qtde char : 78  qtde token :  start_index: 0

Ie ne fay rien 
sans 
Gayeté 
(Montaigne, Des livres) 
Ex Libris 
José Mindlin
-----------------------------------------------------------------------
chunk # 2 2  qtde char : 19  qtde token :  start_index: 0

AS VICTIMAS-ALGOZES
-----------------------------------------------------------------------
chunk # 3 25  qtde char : 154  qtde token :  start_index: 0

AS VICTIMAS-ALGOZES 
QUADROS DA ESCRAVIDÃO 
ROMANCES 
POR 
JOAQUIM M

## 4.3 - Manipulando Documentos HTML

https://python.langchain.com/docs/integrations/document_loaders/web_base

### Links dos HTML

In [100]:
# As Vítimas Algozes
url1 = "https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979"

# A Escrava Isaura
url2 = "https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=92389"

### Carrega os HTMLs

Se ocorrer o **erro** abaixo, instale o Chromedb. Alguma dependência do pacote resolve o problema.

---------------------------------------------------------------------------
IncompleteRead                            Traceback (most recent call last)
/usr/local/lib/python3.10/dist-packages/urllib3/response.py in _error_catcher(self)
    709             try:
--> 710                 yield
    711

15 frames
IncompleteRead: IncompleteRead(821958 bytes read, 2240625 more expected)

The above exception was the direct cause of the following exception:

ProtocolError                             Traceback (most recent call last)
ProtocolError: ('Connection broken: IncompleteRead(821958 bytes read, 2240625 more expected)', IncompleteRead(821958 bytes read, 2240625 more expected))

During handling of the above exception, another exception occurred:

ChunkedEncodingError                      Traceback (most recent call last)
/usr/local/lib/python3.10/dist-packages/requests/models.py in generate()
    816                     yield from self.raw.stream(chunk_size, decode_content=True)
    817                 except ProtocolError as e:
--> 818                     raise ChunkedEncodingError(e)
    819                 except DecodeError as e:
    820                     raise ContentDecodingError(e)

ChunkedEncodingError: ('Connection broken: IncompleteRead(821958 bytes read, 2240625 more expected)', IncompleteRead(821958 bytes read, 2240625 more expected))

In [101]:
# Import das bibliotecas
from langchain.document_loaders import WebBaseLoader

# Cria o carregador da página
carregador = WebBaseLoader([url1, url2])
# carregador.requests_kwargs = {'verify':False}

# Carrega os documentos
documentos = carregador.load()

### Divide e sobrepõe os HTMLs

In [102]:
# Import das bibliotecas
import os
import time
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Parâmetros
chunk_tamanho = 500
chunk_sobreposicao = 100

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_tamanho,
    chunk_overlap  = chunk_sobreposicao, # Número de tokens sobrepostos entre chunks(pedaços)
    length_function = len, # Usa o comprimento do texto como medida de tamanho
    add_start_index = True, #
)

# Guarda o tempo de início
tempo_inicio = time.time()

# Calcula os chunks dos textos
chunks = text_splitter.split_documents(documentos)

tempo_final = time.time()

print(f"Carregando e dividindo {len(texto)} documentos htmls em {tempo_final - tempo_inicio} segundos!")

Carregando e dividindo 1918 documentos htmls em 0.08250641822814941 segundos!


### Mostra os chunks

A sobreposição se encontra ao final e início de cada pedaço(chunk). A variável 'start_index' define onde começa o texto sem a sobreposição.

In [103]:
for i, chunk in enumerate(chunks):
  if i < 20:
    tokens = chunk.page_content.split(" ")
    print('chunk #',i, len(tokens),' qtde char :', len(chunk.page_content),' qtde token :', ' start_index:', chunk.metadata.get('start_index') )
    print()
    print(chunk.page_content)
    print('-----------------------------------------------------------------------')

chunk # 0 25  qtde char : 217  qtde token :  start_index: 16

As vítimas algozes - Joaquim Manuel de Macedo



Fonte: Biblioteca Digital de Literatura de Países Lusófonos

LITERATURA BRASILEIRA 
Textos literários em
meio eletrônico
As
Vítimas-Algozes, de Joaquim Manuel de Macedo
-----------------------------------------------------------------------
chunk # 1 63  qtde char : 442  qtde token :  start_index: 235

Edição de base:
Biblioteca Nacional – setor de obras digitalizadas
ÍNDICE
SIMEÃO, O CRIOULO
PAI- RAIOL, O FEITICEIRO
LUCINDA, A
MUCAMA
CONCLUSÃO
I
SIMEÃO, O CRIOULO
I
No interior e principalmente longe da vila, ou da
freguesia e dos povoados há quase sempre uma venda perto da fazenda: é a
parasita que se apega à árvore; pior que isso, é a inimiga hipócrita que rende
vassalagem à sua vítima.
A venda de que falo é uma taberna especialíssima
-----------------------------------------------------------------------
chunk # 2 74  qtde char : 469  qtde token :  start_index: 604

vassala